**Comentarios generales sobre este notebook:**

- La estructura general es la siguiente:
  - Instalación e importación de librerías.
  - Definición de variables.
  - Obtener el JWT para levantar los datos del campus.
  - Definición de funciones (todas las celdas que dicen "Setup").
  - Ejecución de código de limpieza y transformaciones.
  - Ejecución código para mostrar respuestas numéricas.
  - Ejecución código para mostrar respuestas categóricas.
  - Ejecución código para mostrar correlaciones.
  - Ejecución código para mostrar comentarios o sugerencias.
  - Ejecución de código para mostrar todos los registros a partir del filtrado de algún valor de los datos.
- En la celda que contiene CONSTANTES A DEFINIR EN CADA EJECUCIÓN DEL CÓDIGO, si estas variables se dejan vacías, más adelante en las celdas que contengan la palabra **Interactivo** se van a poder definir (excepto DUMMY_GPT). Esto es porque es puede resultar más práctico completarlas desde ahí en vez de definirlo previamente en código.
- En la celda que contiene CONSTANTES PREDEFINIDAS, esas son constantes también necesarias para que funcione el notebook. Solo habría que modificar en caso que se modifique algún aspecto de las encuestas.
- La pregunta *'Si asististe a clases en vivo: ¿Te gusta el formato de esas clases? Siendo 1 "No me gustan mucho" y 5 "Me gustan mucho":'* tiene una opción en texto (*'No asistí'*) y el resto de las opciones numéricas. Se va a tratar tanto como pregunta de respuesta numérica como de respuesta categórica, considerando en el bloque de respuestas numéricas solo los números y en el bloque de respuestas categóricas todas las opciones.
- Se utiliza GPT solo en la celda de Comentarios o sugerencias. La función que se ejecuta en esa celda tiene un parámetro para definir si usarlo o no (es la constante DUMMY_GPT).


In [1]:
#@markdown Instalación e importación de librerías

!pip install --upgrade openai

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, Layout, VBox, Label
from IPython.display import display, HTML, Markdown
from datetime import date
import babel.dates as dates
import itertools
from scipy.stats import pearsonr, spearmanr
import base64
import requests
from getpass import getpass
import re
from lxml import etree
import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
# CONSTANTES A DEFINIR EN CADA EJECUCIÓN DEL CÓDIGO

CLIENT_ID_CAMPUS = ""
CLIENT_SECRET_CAMPUS = ""
GPT_API_KEY = "" # (Solo se usa GPT en la sección de "Comentarios o sugerencias")
FILTRO_CURSO = "" # Ejemplos de opciones: 'python10.10.23', 'data7.9.23', etc.
FILTRO_ENCUESTA = "" # Opciones: 'Encuesta de Feedback parcial', 'Encuesta de Feedback final', 'Encuesta de Feedback sobre el Examen Final'
FILTRO_FECHAINI = "" # Rango de fecha inferior en formato '%Y-%m-%d'
FILTRO_FECHAFIN = "" # Rango de fecha superior en formato '%Y-%m-%d'
DUMMY_GPT = True # Si está en True, imprime el prompt sin enviar a GPT. Si está en False, no imprime el prompt e imprime los resultados devueltos de GPT.

# CONSTANTES PREDEFINIDAS

CAMPUS_BASE = 'https://estudiantes.campus.humai.com.ar'
MAX_PROMPT_GPT = 10000

TIPOS_DE_ENCUESTA = [
    'Encuesta de Feedback parcial',
    'Encuesta de Feedback final',
    'Encuesta de Feedback sobre el Examen Final'
]

# Lista de emails a filtrar
EMAILS_A_FILTRAR = [
    'licsophiemizrahi@gmail.com',
    'lenasofiafrattini@gmail.com',
    'admin@humai.com.ar',
    'sebastianmeschhenriques@gmail.com'
]

# Diccionario con las configuraciones adicionales de las preguntas
DICT_CONFIGURACION_PREGUNTAS = {
    '¿Qué tan difícil te está pareciendo el material presentado durante la cursada? Siendo 1 "Muy fácil" y 5 "Muy difícil":': {
        'pregunta_corta': 'Dificultad del material',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [1, 2, 3, 4, 5]
    },
    'Si asististe a clases en vivo: ¿Te gusta el formato de esas clases? Siendo 1 "No me gustan mucho" y 5 "Me gustan mucho":': {
        'pregunta_corta': 'Gusto por el formato de las clases en vivo',
        'tipo_pregunta': 'numérica y categórica',
        'orden_opciones': ['No asistí', 1, 2, 3, 4, 5]
    },
    '¿Te gusta el formato de los videos asincrónicos con materiales y ejercicios? Siendo 1 "No me gustan mucho" y 5 "Me gustan mucho":': {
        'pregunta_corta': 'Gusto por el formato de videos asinc, materiales y ejercicios',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [1, 2, 3, 4, 5]
    },
    '¿Qué tan probable es que recomiendes los cursos de Humai a una persona conocida? Siendo 0 "nada probable" y 10 "muy probable":': {
        'pregunta_corta': 'Recomendación de Humai a una persona conocida',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    },
    '¿Cuántas horas en promedio le estás dedicando a cada clase aproximadamente? (Teniendo en cuenta las clases asincrónicas, las clases sincrónicas y la ejercitación)': {
        'pregunta_corta': 'Horas promedio dedicadas por clase',
        'tipo_pregunta': 'categórica',
        'orden_opciones': ['Entre 1 y 2 horas semanales', 'Entre 2 y 4 horas semanales', 'Entre 4 y 6 horas semanales', 'Entre 6 y 8 horas semanales', 'Más de 8 horas semanales']
    },
    'Otros comentarios o sugerencias:': {
        'pregunta_corta': 'Comentarios o sugerencias',
        'tipo_pregunta': 'texto',
        'orden_opciones': ''
    },
    '¿Qué tan difícil te pareció el material presentado durante la cursada? Siendo 1 "Muy fácil" y 5 "Muy difícil":': {
        'pregunta_corta': 'Dificultad del material',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [1, 2, 3, 4, 5]
    },
    'Si asististe a clases en vivo: ¿Te gustó el formato de esas clases? Siendo 1 "No me gustó mucho" y 5 "Me gustó mucho":': {
        'pregunta_corta': 'Gusto por el formato de las clases en vivo',
        'tipo_pregunta': 'numérica y categórica',
        'orden_opciones': ['No asistí', 1, 2, 3, 4, 5]
    },
    '¿Te gustó el formato de los videos asincrónicos con materiales y ejercicios? Siendo 1 "No me gustó mucho" y 5 "Me gustó mucho":': {
        'pregunta_corta': 'Gusto por el formato de videos asinc, materiales y ejercicios',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [1, 2, 3, 4, 5]
    },
    '¿Qué tan valioso te pareció el foro en Discord? Siendo 1 "No me aportó nada" y 5 "Me aportó mucho":': {
        'pregunta_corta': 'Aporte de Discord',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [1, 2, 3, 4, 5]
    },
    '¿Qué tan valioso te pareció el Campus Virtual? Siendo 1 "No me aportó nada" y 5 "Me aportó mucho":': {
        'pregunta_corta': 'Aporte del Campus Virtual',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [1, 2, 3, 4, 5]
    },
    '¿Cuántas horas en promedio le dedicaste a cada clase aproximadamente? (Teniendo en cuenta las clases asincrónicas, las clases sincrónicas y la ejercitación)': {
        'pregunta_corta': 'Horas promedio dedicadas por clase',
        'tipo_pregunta': 'categórica',
        'orden_opciones': ['Entre 1 y 2 horas semanales', 'Entre 2 y 4 horas semanales', 'Entre 4 y 6 horas semanales', 'Entre 6 y 8 horas semanales', 'Más de 8 horas semanales']
    },
    'Otros comentarios y sugerencias:': {
        'pregunta_corta': 'Comentarios y sugerencias',
        'tipo_pregunta': 'texto',
        'orden_opciones': ''
    },
    'Es útil para futuras personas interesadas en estudiar en Humai leer testimonios de ex-alumn@s, ¿aceptás que compartamos tu testimonio?': {
        'pregunta_corta': '¿Compartió testimonio?',
        'tipo_pregunta': 'categórica',
        'orden_opciones': ['No', 'Si']
    },
    '¿Qué tan probable es que recomiendes los cursos de Humai a una persona conocida? Siendo 0 "Nada probable" y 10 "Muy probable":': {
        'pregunta_corta': 'Recomendación de Humai a una persona conocida',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    },
    '¿Qué tan difícil te pareció el Examen Final? Siendo 1 "Muy fácil" y 5 "Muy difícil":': {
        'pregunta_corta': 'Dificultad del Examen Final',
        'tipo_pregunta': 'numérica',
        'orden_opciones': [1, 2, 3, 4, 5]
    },
    'Dejanos tus comentarios sobre el examen a continuación:': {
        'pregunta_corta': 'Comentarios sobre el Examen',
        'tipo_pregunta': 'texto',
        'orden_opciones': ''
    }

}
DF_CONFIGURACION_PREGUNTAS = pd.DataFrame(DICT_CONFIGURACION_PREGUNTAS).T.reset_index().rename(columns = {'index': 'pregunta'})

In [ ]:
#@markdown Obtener claves de Campus y GPT (**Interactivo**)
if not CLIENT_ID_CAMPUS:
  CLIENT_ID_CAMPUS = getpass('Client id: ')
if not CLIENT_SECRET_CAMPUS:
  CLIENT_SECRET_CAMPUS = getpass('Client secret: ')
if not GPT_API_KEY:
  GPT_API_KEY = getpass('Ingresar API Key de OpenAI: ')

In [ ]:

credential = f"{CLIENT_ID_CAMPUS}:{CLIENT_SECRET_CAMPUS}"
encoded_credential = base64.b64encode(credential.encode("utf-8")).decode("utf-8")

#@markdown Obtener JWT token
headers = {"Authorization": f"Basic {encoded_credential}", "Cache-Control": "no-cache"}
data = {"grant_type": "client_credentials", "token_type": "jwt"}

r = requests.post(
    f"{CAMPUS_BASE}/oauth2/access_token", headers=headers, data=data
)
access_token = r.json()["access_token"]

In [ ]:
#@markdown Setup Inscripciones: Funciones relacionadas a la obtención y tranformación de datos de cursos y de estudiantes. Gran parte copiado de **Campus - Inscripciones y cuestionarios.ipynb**
def obtener_users_info(usernames: list, access_token: str):
    headers={"Authorization": f"JWT {access_token}"}
    respuestas = list()

    params = {
        'username': ",".join(usernames)
    }
    r = requests.get(f'{CAMPUS_BASE}/api/user/v1/accounts', params=params, headers=headers)

    if r.ok:
        # print('Info de los usuarios')
        return r.json()
    else:
        raise ValueError(f'Error: no se pudo obtener la informacion de {len(usernames)} usuarios')


def limpiar_user_info(user_info: list, know_columns: bool = False):
    """
    La api de accounts devuelve las siguientes columnas:
    """

    if know_columns and len(user_info) > 0:
        print('Columns disponibles:', user_info[0].keys())

    columns_used = ['username', 'email', 'country', 'last_login', 'is_active']
    df_user = pd.DataFrame(user_info)[columns_used]

    return df_user

def obtener_user_info(member_email: str, access_token: str):
    headers={"Authorization": f"JWT {access_token}"}
    respuestas = list()

    params = {
        'email': member_email
    }
    r = requests.get(f'{CAMPUS_BASE}/api/user/v1/accounts', params=params, headers=headers)

    if r.ok:
        print(f'Info de {member_email}')
        return r.json()[0] if len(r.json()) == 1 else r.json()
    elif r.status_code == 404:
        print(f'Se obtuvo status code {r.status_code}: El usuario {member_email} no existe')
    else:
        raise ValueError(f'Error: no se pudo obtener la informacion de {member_email}.')

def obtener_inscripciones(username: str = None, course_id: str = None, access_token: str = ''):
    headers={"Authorization": f"JWT {access_token}"}
    data = list()

    params = {
        'page_size': 300
    }
    if username is not None:
        params['username'] = username

    if course_id is not None:
        params['course_id'] = course_id

    r = requests.get(f'{CAMPUS_BASE}/api/enrollment/v1/enrollments', params=params, headers=headers)

    if r.ok:
        print(f'Inscripciones status: {r.status_code}, next: {r.json()["next"]}')
        data.extend(r.json()['results'])
        while r.json()["next"]:
            r = requests.get(r.json()["next"], params=params, headers=headers)
            if r.ok:
                data.extend(r.json()['results'])
        return data
    else:
        raise ValueError(f'Error: no se pudo obtener la informacion de inscripciones.')

def obtener_courses_id(access_token: str):
    headers={"Authorization": f"JWT {access_token}"}
    course_info = list()
    r = requests.get(f'{CAMPUS_BASE}/api/courses/v1/courses/', headers=headers)

    if r.ok:
        course_info.extend(r.json()['results'])
        while (new_url := r.json()["pagination"].get("next")):
            r = requests.get(new_url, headers=headers)
            if r.ok:
                course_info.extend(r.json()['results'])
        # print(f'Informacion cursos, next: {r.json()["pagination"].get("next")}')
        return course_info
    else:
        raise ValueError(f'Error: {r.text}.')

def obtener_course_data(course_id: str, access_token: str):
    headers={"Authorization": f"JWT {access_token}"}
    respuestas = list()

    params = {
        'page_size': 300
    }
    r = requests.get(f'{CAMPUS_BASE}/api/enrollment/v1/course/{course_id}', params=params, headers=headers)

    if r.ok:
        print(f'Informacion curso {course_id}')
        return r.json()
    else:
        raise ValueError(f'Error: no se pudo obtener la informacion del curso {course_id}.')

def inscripciones_curso(course_id: str, access_token: str):
    data_course = obtener_inscripciones(course_id=course_id, access_token=access_token)
    df = pd.DataFrame(data_course).rename({'is_active': 'is_active_course'}, axis=1)

    user_data = obtener_users_info(df['user'].unique(), access_token)
    df_user = limpiar_user_info(user_data).rename({'is_active': 'is_active_user'}, axis=1)

    return df.merge(df_user, how='left', left_on='user', right_on='username').drop('user', axis=1)

def obtener_listado_cursos(access_token):
  courses_data = obtener_courses_id(access_token)
  df_courses = pd.DataFrame(courses_data)
  df_courses['code'] = df_courses.course_id.apply(lambda x: x.split('+')[-1])
  listado_cursos = df_courses.sort_values(by='code')['code'].to_list()
  return listado_cursos

In [ ]:
#@markdown Setup Encuestas: Funciones relacionadas a la obtención y tranformación de las preguntas y respuestas de las encuestas. Gran parte copiado de **Campus - Inscripciones y cuestionarios.ipynb**
def obtener_cuestionario(code: str, access_token: str, keyname: str = None):
    headers={"Authorization": f"JWT {access_token}"}
    respuestas = list()

    if keyname is None or len(keyname) == 0:
        keyname = re.search(r'^\D+', code)[0]

    r = requests.get(f'{CAMPUS_BASE}/api/grades/v1/submission_history/course-v1%3AHumai%2B{keyname}%2B{code}/', headers=headers)

    if not r.ok:
        print('Se genero un erro en la primera iteracion')
        print(r.text)
        return []

    if len(r.json()['results']) == 0:
        raise ValueError(f"No se pudo obtener datos del curso {keyname}+{code}.\
            Revisar el keyname o code asignado en el campus"
        )

    respuestas += r.json()['results']
    while r.json().get('next') is not None:
        r = requests.get(r.json()['next'], headers=headers)

        if r.ok:
            respuestas += r.json()['results']

    return respuestas

def ordenar_y_expandir(dic):
    try:
        sorted_dict = dict(sorted(dic.items(), key=lambda x: int(x[0].split('_')[0])))
    except Exception as e:
        sorted_dict = dict(sorted(dic.items()))
        print(f'Error en ordenar y expandir: {e}')
    return pd.Series(sorted_dict)

def limpiar_inputs(input_state: dict):
    if input_state is None:
        return input_state

    res = dict()
    for key, value in input_state.items():
        item = re.findall(r'\w+_(\d+)_\d', key)
        if item:
            res[item[0]] = value

    return res


def limpiar_cuestionario(respuestas: list):
    clean = list()
    t = dict()
    for item in respuestas:
        for problem in item['problems']:
            for submition in problem['submission_history']:
                clean.append({
                    'user': item['user'],
                    'class_name': problem['name'],
                    'block_id': problem['location'],
                    'grade': submition.get('grade', {}),
                    'points_earned': submition['state'].get('score', 0).get('raw_earned'),
                    'points_possible': submition['state'].get('score', 0).get('raw_possible'),
                    'respuestas': limpiar_inputs(submition['state'].get('student_answers')),
                    'datetime': submition['state'].get('last_submission_time'),
                    'raw': problem.get('data'),
                })
    df = pd.DataFrame(clean)
    # print('Check grade None con nota:', df[df['grade'].isna() & (df['points_earned'] != 0)].shape[0])
    df = df[df['grade'].notna()].reset_index(drop=True)
    df['nota'] = (df['points_earned'] * 10 / df['points_possible']).round(1)
    df['type'] = df['block_id'].str.extract(r'type@(\w+)')
    return df

def extraer_respuestas(df_, clase, choice_content: bool = False, choice_question: bool = False):
    df = df_[df_.class_name == clase].copy()
    return df.join(
        df.apply(clean_responses, axis=1, choice_content=choice_content, choice_question=choice_question)
    ).sort_values('datetime')


def clean_whitespaces(text):
    return re.sub(r'\s+', ' ', text.replace('\n', ''))

def get_questions(raw_string: str):
    content = etree.fromstring(raw_string)
    groups = content.xpath('//stringresponse | //multiplechoiceresponse | //choiceresponse')
    quiz = dict()
    for idx, group in enumerate(groups, start=1):
        try:
            preguntas = group.xpath('//Strong | .//label[not(Strong)]')
            for idx2, pregunta in enumerate(preguntas):


              text = clean_whitespaces(pregunta.text)
              num, title = re.findall(r"(\d+)?[).]?\s*(.+)", text)[0]

              if len(num) < 1:
                  num = idx + idx2
              quiz[int(num)] = {
                  "title": title,
              }
              if pregunta.tag != 'Strong':
                for n_choice, choice in enumerate(group.xpath('.//choice')):
                    quiz[int(num)][f'choice_{n_choice}'] = clean_whitespaces(choice.text), choice.attrib.get("correct") == "true"

        except Exception as e:
            for idx2, pregunta in enumerate(preguntas):
              print(f"Error {e} en:\n{pregunta.text}")

    return quiz

def clean_responses(row, choice_content: bool = False, choice_question: bool = False):
    index_choices = 0 if choice_content else 1
    responses = dict()
    quiz = get_questions(row['raw'])
    for k, v in row['respuestas'].items():
        question = int(k.split('_')[0]) - 1
        q = quiz[question]
        if isinstance(v, str) and "choice_" not in v:
            # Respuesta de texto libre
            responses[str(question)] = v
            responses[f'{question}_question'] = q["title"]

            continue
        try:

            if isinstance(v, list):
                responses[str(question)] = [q[choice][index_choices] for choice in v]
                if choice_question:
                    responses[f'{question}_question'] = q["title"]
            else:
                response = v.split('_')[1]
                try:
                    responses[str(question)] = q[v][index_choices]
                except KeyError:
                    print("KeyError: La clave", v, "no está en el diccionario. Saltando esta iteración...")
                    continue  # Salta a la siguiente iteración del bucle


                if choice_question:
                    responses[f'{question}_question'] = q["title"]
        except IndexError:
            print(len(quiz), int(question), int(response))

    return pd.Series(ordenar_y_expandir(responses))

def filtrar_y_eliminar_filas(df, seleccion_tipo_encuesta_value):
    # Filtra el DataFrame según el valor seleccionado
    df_filtrado = df[df['class_name'] == seleccion_tipo_encuesta_value]


    # Si después de filtrar por curso el dataframe está vacío, lo devuelvo así y lo valido cuando se llama a esta función
    if df_filtrado.empty:
      return df_filtrado
    # Ordena el DataFrame filtrado por la columna 'datetime'
    df_filtrado = df_filtrado.sort_values(by=['datetime'], ascending=False)


    # Aplica la función 'get_questions' a la columna 'raw'
    series_check_preguntas = df_filtrado['raw'].apply(get_questions)

    # Calcula el límite
    limite = len(series_check_preguntas.iloc[0])

    # Elimina las filas según la condición
    filtro = df_filtrado['respuestas'].apply(lambda respuestas: len(respuestas) != limite)
    df_final = df_filtrado[~filtro]
    print(f"Formularios únicos: {df_filtrado['raw'].nunique()}")
    print(f"Total respuestas original: {df_filtrado.shape[0]}")
    print(f"Total respuestas después del filtrado por no coincidencia de respuestas con la encuesta: {df_final.shape[0]}")

    return df_final

def reemplazar_columnas_por_preguntas(df):

  # Obtener las preguntas y sus respuestas
  questions = []
  answers = []

  for col in df.columns:
      if col.endswith('_question'):
          question_number = col.split('_')[0]
          question = df[col].iloc[0]  # Tomar la pregunta de la primera fila
          answer_col = question_number  # Columna de respuesta
          questions.append(question)
          answers.append(answer_col)

  # Seleccionar automáticamente las columnas que son strings y no comienzan con un número
  string_columns = [col for col in df.columns if not col[0].isdigit()]

  # Crear un nuevo DataFrame con las columnas seleccionadas
  new_df = pd.DataFrame(columns=string_columns + questions)

  # Llenar el nuevo DataFrame con las respuestas
  for _, row in df.iterrows():
      user_info = row[string_columns]
      answers_row = [row[col] for col in answers]
      new_row = pd.concat([user_info, pd.Series(answers_row, index=questions)])
      new_df = pd.concat([new_df, new_row.to_frame().T], ignore_index=True)

  return new_df

In [ ]:
#@markdown Setup Gráficos: Funciones que adaptan la información de los dataframes recibidos para poder generar los distintos gráficos de respuestas numéricas y categóricas, generar también gráficos de correlación y mostrar respuestas de texto.

def graficar_respuestas_numericas(df_preguntas_todas, fecha_ini, fecha_fin):
  lista_preguntas_opciones_numericas = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['tipo_pregunta'] == 'numérica']['pregunta'].values
  columnas_preguntas_opciones_numericas_y_categoricas = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['tipo_pregunta'] == 'numérica y categórica']['pregunta'].values
  df_preguntas_todas_filtradas = df_preguntas_todas[(pd.to_datetime(df_preguntas_todas.index) >= fecha_ini) & (pd.to_datetime(df_preguntas_todas.index) < fecha_fin)]
  lista_numericas_presentes = list(set(df_preguntas_todas_filtradas.columns).intersection(lista_preguntas_opciones_numericas))
  lista_numericas_y_categoricas_presentes = list(set(df_preguntas_todas_filtradas.columns).intersection(columnas_preguntas_opciones_numericas_y_categoricas))
  dict_rename_preguntas = dict(zip(DF_CONFIGURACION_PREGUNTAS['pregunta'], DF_CONFIGURACION_PREGUNTAS['pregunta_corta']))

  df_encuesta_parcial_respuestas_numericas_ts = df_preguntas_todas_filtradas[np.append(lista_numericas_presentes, lista_numericas_y_categoricas_presentes)].copy()
  df_encuesta_parcial_respuestas_numericas_ts.rename(columns=dict_rename_preguntas, inplace=True)
  df_encuesta_parcial_respuestas_numericas_ts.sort_index(inplace=True)
  elementos_numericos_y_categoricos = []

  for elemento_numerico_y_categorico in lista_numericas_y_categoricas_presentes:
    elemento_renombrado = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['pregunta'] == elemento_numerico_y_categorico]['pregunta_corta'].values[0]
    elementos_numericos_y_categoricos.append(elemento_renombrado)
    # Para este bloque de respuestas numéricas, a los que sean de tipo numéricas y categóricas solo tomamos la parte numérica. Lo que no sea número se convierte en NaN.
    df_encuesta_parcial_respuestas_numericas_ts[elemento_renombrado] = pd.to_numeric(df_encuesta_parcial_respuestas_numericas_ts[elemento_renombrado], errors='coerce')

  # df_encuesta_parcial_respuestas_numericas_y_categoricas_ts = df_encuesta_parcial_respuestas_numericas_ts[elementos_numericos_y_categoricos].dropna().astype('Int64')
  df_encuesta_parcial_respuestas_numericas_ts = df_encuesta_parcial_respuestas_numericas_ts.astype('Int64')
  # df_encuesta_parcial_respuestas_numericas_ts = df_encuesta_parcial_respuestas_numericas_ts.drop(columns=elementos_numericos_y_categoricos)


  ######################## DISTRIBUCIÓN A LO LARGO DEL TIEMPO ########################

  dict_respuestas_totales = {}

  # Definir el número de columnas para los subplots
  columnas_subplot = min(len(df_encuesta_parcial_respuestas_numericas_ts.columns), 2)

  # Calcular el número de filas necesario
  filas_subplot = int(np.ceil(len(df_encuesta_parcial_respuestas_numericas_ts.columns) / columnas_subplot))

  # Crear subplots con Plotly Graph Objects
  fig = make_subplots(max(filas_subplot,1), columnas_subplot)

  suma_idx = 0
  for idx, columna in enumerate(df_encuesta_parcial_respuestas_numericas_ts.columns):
      suma_idx = idx
      # Calcular la fila y la columna para el subplot actual
      fila = idx // columnas_subplot + 1
      columna_subplot = idx % columnas_subplot + 1
      df_encuesta_parcial_respuestas_numericas_ts
      # Agregar el gráfico de barras
      fig.add_trace(go.Scatter(x=df_encuesta_parcial_respuestas_numericas_ts.index,
                          y=df_encuesta_parcial_respuestas_numericas_ts[columna],
                          name=columna,
                          mode='markers'),
                    row=fila, col=columna_subplot)

      # Calcular el promedio y agregar la línea promedio
      promedio = df_encuesta_parcial_respuestas_numericas_ts[columna].mean()

      if pd.isna(promedio):
        promedio = 0

      fig.add_trace(go.Scatter(x=df_encuesta_parcial_respuestas_numericas_ts.index,
                              y=np.round([promedio] * len(df_encuesta_parcial_respuestas_numericas_ts), 2),
                              mode='lines',
                              line=dict(color='red', width=2),
                              name=f'Promedio: {promedio:.2f}',
                              showlegend=False),
                    row=fila, col=columna_subplot)

      dict_respuestas_totales[columna] = df_encuesta_parcial_respuestas_numericas_ts[columna].dropna().shape[0]

  # Actualizar el diseño de la figura y mostrarla
  fig.update_layout(
      width=1250,
      height=800,
      title='Gráficos de respuestas numéricas',
      showlegend=True
  )

  for pregunta, cantidad in dict_respuestas_totales.items():
    print(f"Respuestas totales {pregunta}: {cantidad}")

  fig.show()



  ######################## HISTOGRAMAS ########################



  dict_respuestas_totales = {}

  # Definir el número de columnas para los subplots
  columnas_subplot = min(len(df_encuesta_parcial_respuestas_numericas_ts.columns), 2)

  # Calcular el número de filas necesario
  filas_subplot = int(np.ceil(len(df_encuesta_parcial_respuestas_numericas_ts.columns) / columnas_subplot))

  # Crear subplots con Plotly Graph Objects
  fig = make_subplots(max(filas_subplot,1), columnas_subplot)

  suma_idx = 0
  for idx, columna in enumerate(df_encuesta_parcial_respuestas_numericas_ts.columns):
      suma_idx = idx
      # Calcular la fila y la columna para el subplot actual
      fila = idx // columnas_subplot + 1
      columna_subplot = idx % columnas_subplot + 1


      # Verificar si todos los valores únicos pueden convertirse en números
      todos_los_elementos_numericos = not df_encuesta_parcial_respuestas_numericas_ts[columna].dropna().empty and pd.to_numeric(df_encuesta_parcial_respuestas_numericas_ts[columna].dropna(), errors='coerce').notnull().all()


      # Verificar si la serie 'valor' es de tipo entero (int)
      if todos_los_elementos_numericos:
          # display(df_encuesta_parcial_respuestas_categoricas_ts)
          # display(df_encuesta_parcial_respuestas_categoricas_ts[columna])
          df_encuesta_parcial_respuestas_numericas_ts[columna] = df_encuesta_parcial_respuestas_numericas_ts[columna].astype('Int64')

          # Obtener todos los valores únicos que podrían aparecer en el histograma
          valores_posibles = list(range(df_encuesta_parcial_respuestas_numericas_ts[columna].min(), df_encuesta_parcial_respuestas_numericas_ts[columna].max() + 1))

          # Contar las ocurrencias de cada valor
          conteo_valores = df_encuesta_parcial_respuestas_numericas_ts[columna].value_counts().reindex(valores_posibles, fill_value=0)

      else:
            # En caso de que la serie no sea de tipo entero, se obtienen todos los valores únicos
            valores_posibles = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['pregunta_corta'] == columna]['orden_opciones'].values[0]
            valores_posibles = [str(elemento) for elemento in valores_posibles]

            # Contar las ocurrencias de cada valor
            conteo_valores = df_encuesta_parcial_respuestas_numericas_ts[columna].value_counts().reindex(valores_posibles).dropna()

      # Calcular los porcentajes
      porcentajes = (conteo_valores / conteo_valores.sum()) * 100

      # Agregar el gráfico de barras
      fig.add_trace(
        go.Histogram(
          x=df_encuesta_parcial_respuestas_numericas_ts[columna],
          name=columna,
          text=[f'{valor_absoluto}<br>({porcentaje:.2f}%)' for valor_absoluto, porcentaje in zip(conteo_valores.values, porcentajes)],
          xbins=dict(
            start=df_encuesta_parcial_respuestas_numericas_ts[columna].min() - 0.5,
            end=df_encuesta_parcial_respuestas_numericas_ts[columna].max() + 0.5,
            size=1
          )
        ),
        row=fila,
        col=columna_subplot
      )

  # Actualizar el diseño de la figura y mostrarla
  fig.update_layout(
      width=1250,
      height=800,
      title='Histogramas de respuestas numéricas',
      bargap=0.1,
      showlegend=True
  )


  fig.show()



def graficar_respuestas_categoricas(df_preguntas_todas, fecha_ini, fecha_fin):

  lista_preguntas_opciones_categoricas = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['tipo_pregunta'] == 'categórica']['pregunta'].values
  columnas_preguntas_opciones_numericas_y_categoricas = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['tipo_pregunta'] == 'numérica y categórica']['pregunta'].values
  df_preguntas_todas_filtradas = df_preguntas_todas[(pd.to_datetime(df_preguntas_todas.index) >= fecha_ini) & (pd.to_datetime(df_preguntas_todas.index) < fecha_fin)]
  lista_categoricas_presentes = list(set(df_preguntas_todas_filtradas.columns).intersection(lista_preguntas_opciones_categoricas))
  lista_numericas_y_categoricas_presentes = list(set(df_preguntas_todas_filtradas.columns).intersection(columnas_preguntas_opciones_numericas_y_categoricas))
  dict_rename_preguntas = dict(zip(DF_CONFIGURACION_PREGUNTAS['pregunta'], DF_CONFIGURACION_PREGUNTAS['pregunta_corta']))

  df_encuesta_parcial_respuestas_categoricas_ts = df_preguntas_todas_filtradas[np.append(lista_categoricas_presentes, lista_numericas_y_categoricas_presentes)].copy()
  df_encuesta_parcial_respuestas_categoricas_ts.rename(columns=dict_rename_preguntas, inplace=True)
  df_encuesta_parcial_respuestas_categoricas_ts.sort_index(inplace=True)
  elementos_numericos_y_categoricos = []

  for elemento_numerico_y_categorico in lista_numericas_y_categoricas_presentes:
    elemento_renombrado = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['pregunta'] == elemento_numerico_y_categorico]['pregunta_corta'].values[0]
    elementos_numericos_y_categoricos.append(elemento_renombrado)

  if len(df_encuesta_parcial_respuestas_categoricas_ts.columns) != 0:

    ######################## DISTRIBUCIÓN A LO LARGO DEL TIEMPO ########################

    dict_respuestas_totales = {}

    # Definir el número de columnas para los subplots
    columnas_subplot = min(len(df_encuesta_parcial_respuestas_categoricas_ts.columns),2)

    # Calcular el número de filas necesario
    filas_subplot = int(np.ceil(len(df_encuesta_parcial_respuestas_categoricas_ts.columns) / columnas_subplot))

    # Crear subplots con Plotly Graph Objects
    fig = make_subplots(max(filas_subplot, 1), columnas_subplot)


    suma_idx = 0
    for idx, columna in enumerate(df_encuesta_parcial_respuestas_categoricas_ts.columns):
        suma_idx = idx
        # Calcular la fila y la columna para el subplot actual
        fila = idx // columnas_subplot + 1
        columna_subplot = idx % columnas_subplot + 1
        # df_encuesta_parcial_respuestas_categoricas_ts_reordenado = df_encuesta_parcial_respuestas_categoricas_ts.reindex(orden_categorias, axis=0)

        # Agregar el gráfico de barras
        fig.add_trace(go.Scatter(
                            x=df_encuesta_parcial_respuestas_categoricas_ts.index,
                            y=df_encuesta_parcial_respuestas_categoricas_ts[columna],
                            name=columna,
                            mode='markers'),
                      row=fila, col=columna_subplot)
        # fig.update_yaxes(categoryorder='category ascending', row=fila, col=columna_subplot)
        fig.update_yaxes(categoryorder='array', categoryarray=DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['pregunta_corta'] == columna]['orden_opciones'].values[0], row=fila, col=columna_subplot)

        dict_respuestas_totales[columna] = df_encuesta_parcial_respuestas_categoricas_ts[columna].dropna().shape[0]

    # Actualizar el diseño de la figura y mostrarla
    fig.update_layout(title='Gráficos de respuestas categóricas',
                      showlegend=True)

    for pregunta, cantidad in dict_respuestas_totales.items():
      print(f"Respuestas totales {pregunta}: {cantidad}")

    fig.show()



    ######################## HISTOGRAMAS ########################



    columnas_subplot = min(len(df_encuesta_parcial_respuestas_categoricas_ts.columns),2)

    # Calcular el número de filas necesario
    filas_subplot = int(np.ceil(len(df_encuesta_parcial_respuestas_categoricas_ts.columns) / columnas_subplot))

    # Crear subplots con Plotly Graph Objects
    fig = make_subplots(max(filas_subplot, 1), columnas_subplot, vertical_spacing=0.2)


    for idx, columna in enumerate(df_encuesta_parcial_respuestas_categoricas_ts.columns):
        suma_idx = idx
        # Calcular la fila y la columna para el subplot actual
        fila = idx // columnas_subplot + 1
        columna_subplot = idx % columnas_subplot + 1

        # Las respuestas que son numéricas y categóricas traen una parte numérica y otra categórica.
        # Si las respuestas son todas numéricas, Plotly lo interpreta así y rellena valores numéricos que existen con cero y eso complica para calcular porcentaje y asignarlo a las barras.
        # Estas líneas lo solucionan

        # Verificar si todos los valores únicos pueden convertirse en números
        todos_los_elementos_numericos = not df_encuesta_parcial_respuestas_categoricas_ts[columna].dropna().empty and pd.to_numeric(df_encuesta_parcial_respuestas_categoricas_ts[columna].dropna(), errors='coerce').notnull().all()

        # Verificar si la serie 'valor' es de tipo entero (int)
        if todos_los_elementos_numericos:
            df_encuesta_parcial_respuestas_categoricas_ts[columna] = df_encuesta_parcial_respuestas_categoricas_ts[columna].astype('Int64')

            # Obtener todos los valores únicos que podrían aparecer en el histograma
            valores_posibles = list(range(df_encuesta_parcial_respuestas_categoricas_ts[columna].min(), df_encuesta_parcial_respuestas_categoricas_ts[columna].max() + 1))

            # Contar las ocurrencias de cada valor
            conteo_valores = df_encuesta_parcial_respuestas_categoricas_ts[columna].value_counts().reindex(valores_posibles, fill_value=0)
        else:

            # En caso de que la serie no sea de tipo entero, se obtienen todos los valores únicos
            valores_posibles = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['pregunta_corta'] == columna]['orden_opciones'].values[0]
            valores_posibles = [str(elemento) for elemento in valores_posibles]

            # Contar las ocurrencias de cada valor
            conteo_valores = df_encuesta_parcial_respuestas_categoricas_ts[columna].value_counts().reindex(valores_posibles).dropna()

            # valores_posibles tiene que tener el orden configurado de los elementos, pero solo los elementos que existan en conteo_valores
            valores_posibles = [elem for elem in valores_posibles if elem in conteo_valores.index.values]

        # Calcular los porcentajes
        porcentajes = (conteo_valores / conteo_valores.sum()) * 100

        # Agregar el gráfico de barras
        fig.add_trace(go.Histogram(

                            x=df_encuesta_parcial_respuestas_categoricas_ts[columna],
                            text=[f'{valor_absoluto}<br>({porcentaje:.2f}%)' for valor_absoluto, porcentaje in zip(conteo_valores.values, porcentajes)],
                            name=columna),
                      row=fila, col=columna_subplot)
        fig.update_xaxes(categoryorder='array', categoryarray=valores_posibles, row=fila, col=columna_subplot)

    # Actualizar el diseño de la figura y mostrarla
    fig.update_layout(
        height=800,
        title='Histogramas de respuestas categóricas',
        showlegend=True,
        bargap=0.1
    )

    fig.show()

  else:
    print("No existen preguntas categóricas")



def graficar_correlaciones(df_preguntas_todas, fecha_ini, fecha_fin):
  lista_preguntas_opciones_texto = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['tipo_pregunta'] == 'texto']['pregunta'].values
  dict_rename_preguntas = dict(zip(DF_CONFIGURACION_PREGUNTAS['pregunta'], DF_CONFIGURACION_PREGUNTAS['pregunta_corta']))
  df_preguntas_todas_filtradas = df_preguntas_todas[(pd.to_datetime(df_preguntas_todas.index) >= fecha_ini) & (pd.to_datetime(df_preguntas_todas.index) < fecha_fin)]
  lista_texto_presentes = list(set(df_preguntas_todas_filtradas.columns).intersection(lista_preguntas_opciones_texto))
  df_preguntas_no_texto_filtradas_rename = df_preguntas_todas_filtradas.drop(columns=lista_texto_presentes).rename(columns=dict_rename_preguntas)
  combinaciones_columnas = list(itertools.combinations(df_preguntas_no_texto_filtradas_rename.columns, 2))

  ######################## HEATMAP ########################
  def correlate(matrix, corr=pearsonr):
      rows, cols = matrix.shape[0], matrix.shape[1]
      r = np.ones(shape=(rows, rows))
      p = np.ones(shape=(rows, rows))
      for i in range(rows):
          for j in range(i+1, rows):
              r_, p_ = corr(matrix[i], matrix[j])
              r[i, j] = r[j, i] = r_
              p[i, j] = p[j, i] = p_

      return r, p

  # Mapear las opciones categóricas a una opción numérica para evaluar las correlaciones.
  # ACLARACIÓN: La opción 'No asistí' fue asignada como cero para que se pueda asociar en el heatmap, pero no es lo mismo no asistir que decir que no le gustó el formato de clases
  df_mapear_a_numero = DF_CONFIGURACION_PREGUNTAS[(DF_CONFIGURACION_PREGUNTAS['tipo_pregunta'] == 'categórica') | (DF_CONFIGURACION_PREGUNTAS['tipo_pregunta'] == 'numérica y categórica')][['pregunta_corta', 'orden_opciones']].drop_duplicates(subset=['pregunta_corta'])

  # Inicializar un diccionario vacío para almacenar los resultados
  mapeo_resultado = {}
  # Iterar sobre las filas del dataframe df_mapear_a_numero
  for index, row in df_mapear_a_numero.iterrows():
      # Crear un diccionario con los valores reales asignados a un número
      mapeo_resultado[row['pregunta_corta']] = {valor: indice for indice, valor in enumerate(row['orden_opciones'])}

  df_preguntas_no_texto_filtradas_rename_cast_numerico = (df_preguntas_no_texto_filtradas_rename.replace(mapeo_resultado)).astype(int)

  correlaciones, pvalues = correlate(df_preguntas_no_texto_filtradas_rename_cast_numerico.values.T)
  fig = go.Figure(data=go.Heatmap(
                      z=correlaciones,
                      x=df_preguntas_no_texto_filtradas_rename_cast_numerico.columns,
                      y=df_preguntas_no_texto_filtradas_rename_cast_numerico.columns,
                      text=correlaciones,
                      texttemplate="%{text:.2f}",
                      hovertext=pvalues,
                      hovertemplate =
                        '<b>%{x} - %{y}</b>'+
                        '<br><b>Correlación de Pearson</b>: %{z:.2f}'+
                        '<br><b>P valor</b>: %{hovertext:.3f}',
                      name='',
                      colorscale='oranges'
                      ))

  fig.update_layout(yaxis=dict(autorange='reversed'))
  fig.show()



  ######################## CORRELACIONES ########################



  # Definir el número de columnas para los subplots
  columnas_subplot = min(len(combinaciones_columnas),2)
  if len(combinaciones_columnas) != 0:
    # Calcular el número de filas necesario
    filas_subplot = int(np.ceil((len(combinaciones_columnas)) / columnas_subplot))

    # Crear subplots con Plotly Graph Objects
    fig = make_subplots(max(filas_subplot, 1), columnas_subplot, column_widths=[0.35, 0.35], horizontal_spacing=0.3)
    for idx, tuplas_columnas in enumerate(combinaciones_columnas):
        suma_idx = idx
        # Calcular la fila y la columna para el subplot actual
        fila = idx // columnas_subplot + 1
        columna_subplot = idx % columnas_subplot + 1

        df_preguntas_no_texto_filtradas_rename_agrupado = df_preguntas_no_texto_filtradas_rename.groupby([tuplas_columnas[0], tuplas_columnas[1]]).size().reset_index(name='Conteo')
        df_preguntas_no_texto_filtradas_rename_agrupado.sort_values(by=[tuplas_columnas[0], tuplas_columnas[1]], inplace=True)

        fig.add_trace(
            go.Scatter(
                x=df_preguntas_no_texto_filtradas_rename_agrupado[tuplas_columnas[0]],
                y=df_preguntas_no_texto_filtradas_rename_agrupado[tuplas_columnas[1]],
                mode='markers',
                text=['Cantidad: ' + str(valor) for valor in df_preguntas_no_texto_filtradas_rename_agrupado['Conteo'].astype(str)],
                marker=dict(
                    size=df_preguntas_no_texto_filtradas_rename_agrupado['Conteo'],
                    sizemode='area',
                    sizeref=2.*max(df_preguntas_no_texto_filtradas_rename_agrupado['Conteo'])/(40.**2),
                    sizemin=4
                ),
                name="",
                showlegend=False
            ),
          row=fila,
          col=columna_subplot
        )

        # ORDEN EJE X e Y

        fig.update_xaxes(categoryorder='array', categoryarray=DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['pregunta_corta'] == tuplas_columnas[0]]['orden_opciones'].values[0], row=fila, col=columna_subplot)
        fig.update_yaxes(categoryorder='array', categoryarray=DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['pregunta_corta'] == tuplas_columnas[1]]['orden_opciones'].values[0], row=fila, col=columna_subplot)

        # Actualizar el diseño de la figura y mostrarla
        fig.update_layout(
          title='Correlaciones entre todas las respuestas',
          showlegend=True
        )
        fig.update_layout(
            height=(430 * filas_subplot),  # Alto de la figura en píxeles
        )
        fig.update_xaxes(title_text=tuplas_columnas[0], row=fila, col=columna_subplot)
        fig.update_yaxes(title_text=tuplas_columnas[1], row=fila, col=columna_subplot)

    fig.show()

  else:
    print("No existen combinaciones de preguntas")


def analisis_comentarios(df_preguntas_todas_con_datos_usuario, fecha_ini, fecha_fin, dummy_gpt):
  client = openai.OpenAI(api_key=GPT_API_KEY)
  llm_model = "gpt-4-turbo-preview"


  # En este caso tomo el dataframe que tiene datos del username y del email para poder imprimirlos en la tabla

  lista_preguntas_opciones_texto = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['tipo_pregunta'] == 'texto']['pregunta'].values
  dict_rename_preguntas = dict(zip(DF_CONFIGURACION_PREGUNTAS['pregunta'], DF_CONFIGURACION_PREGUNTAS['pregunta_corta']))
  df_preguntas_todas_filtradas = df_preguntas_todas_con_datos_usuario[(pd.to_datetime(df_preguntas_todas_con_datos_usuario['datetime']) >= fecha_ini) & (pd.to_datetime(df_preguntas_todas_con_datos_usuario['datetime']) < fecha_fin)]
  columnas_texto_presentes = list(set(df_preguntas_todas_filtradas.columns).intersection(lista_preguntas_opciones_texto))
  df_preguntas_texto_filtradas = df_preguntas_todas_filtradas[np.append(['username', 'email', 'datetime'],columnas_texto_presentes)].copy()
  df_preguntas_texto_filtradas_rename = df_preguntas_texto_filtradas.rename(columns=dict_rename_preguntas).sort_values(by='datetime', ascending=False)


  ############################# ANÁLISIS CON GPT ##########################
  system_prompt = """
    # Tarea: Generar análisis de los comentarios de la encuesta.

    ## Comentarios:
    """

  # Iterar sobre las filas del DataFrame y agregar los comentarios y fechas al prompt
  for index, row in df_preguntas_texto_filtradas_rename.iterrows():
      comentario = row[dict_rename_preguntas[columnas_texto_presentes[0]]]
      fecha = row['datetime']
      if not comentario:
        comentario = "-"
      system_prompt += f"- Fecha: {fecha}, Comentario: {comentario}\n"

    # Solicitar el resumen y análisis
  system_prompt += """
    ## Resumen y Análisis:
    Por favor, generá un análisis de los comentarios anteriores empezando con contabilizar el total y siguiendo con la proporción en número absoluto y en porcentaje de las distintas categorías de comentarios encontrados.
    Las categorías pueden dividirse en: Comentarios vacíos o sin valor, Comentarios positivos, Comentarios negativos y Comentarios con sugerencias. Pueden agregarse más categorías si lo considerás.
    Finalmente analizar la tendencia de esas categorías a través del tiempo.

    Considerar cómo comentarios a todo lo que tenga una fecha aunque estén vacíos o tenga guiones, puntos, símbolos o letras que no conformen una palabra con sentido.

    La estructura de respuesta tiene que ser:
    1. Total de comentarios
    2. Proporciones de las distintas categorías de comentarios.
    3. Tendencia de las categorías de comentarios a través del tiempo.

    Que los resultados sean un resumen, no dar tantos detalles. Al mostrar las cantidades de comentarios de las distintas categorías hacerlo con formato {[numero_absoluto] ([porcentaje]%)}
    """

  print(f"Cantidad máxima de carácteres seteado en prompt: {MAX_PROMPT_GPT}")
  print(f"Carácteres del prompt a enviar: {len(system_prompt)}")

  if len(system_prompt) > MAX_PROMPT_GPT:
    raise Exception('Prompt demasiado largo, revisar')
  if dummy_gpt:
      print("No se hace análisis con GPT porque se seteó dummy_gpt en True. Se muestra el prompt que se hubiese enviado:")
      display(Markdown(system_prompt))
  else:
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_prompt},
        ],
    )
    display(Markdown(response.choices[0].message.content))


  ############################# LISTAR COMENTARIOS ##########################
  fig = go.Figure(data=[go.Table(
      header=dict(values=list(df_preguntas_texto_filtradas_rename.columns),
                  fill_color='#F59410',
                  align='left'),
      cells=dict(values=[
                df_preguntas_texto_filtradas_rename['username'],
                df_preguntas_texto_filtradas_rename['email'],
                df_preguntas_texto_filtradas_rename['datetime'],
                df_preguntas_texto_filtradas_rename[dict_rename_preguntas[columnas_texto_presentes[0]]].values.flatten()],
                fill_color='#FCDCB1',

                align=['left','center']
                ))
  ])
  fig.update_layout(title='Comentarios del período seleccionado ordenado por fecha')
  fig.show()


def filtrar_registros_completos(df_preguntas_todas_con_datos_usuario, dict_filtros_valores_campos, fecha_ini, fecha_fin):
  filtro = None
  dict_rename_preguntas = dict(zip(DF_CONFIGURACION_PREGUNTAS['pregunta'], DF_CONFIGURACION_PREGUNTAS['pregunta_corta']))
  df_preguntas_todas_con_datos_usuario = df_preguntas_todas_con_datos_usuario[(pd.to_datetime(df_preguntas_todas_con_datos_usuario['datetime']) >= fecha_ini) & (pd.to_datetime(df_preguntas_todas_con_datos_usuario['datetime']) < fecha_fin)]
  df_preguntas_todas_con_datos_usuario_renamed = df_preguntas_todas_con_datos_usuario.rename(columns=dict_rename_preguntas)
  for columna in dict_filtros_valores_campos:
      valor_filtro = dict_filtros_valores_campos[columna].value
      if valor_filtro != "":
          if filtro is None:
              filtro = df_preguntas_todas_con_datos_usuario_renamed[columna].str.contains(valor_filtro, case=False)
          else:
              filtro &= df_preguntas_todas_con_datos_usuario_renamed[columna].str.contains(valor_filtro, case=False)
  if filtro is not None:
    df_preguntas_todas_con_datos_usuario_renamed = df_preguntas_todas_con_datos_usuario_renamed[filtro]

  fig = go.Figure(data=[go.Table(
      header=dict(values=list(df_preguntas_todas_con_datos_usuario_renamed.columns),
                  fill_color='#F59410',
                  align='left'),
      cells=dict(values=[df_preguntas_todas_con_datos_usuario_renamed[column] for column in df_preguntas_todas_con_datos_usuario_renamed.columns],
                fill_color='#FCDCB1',

                align=['left','center']
                ))
  ])

  fig.update_layout(
      width = (230 * len(df_preguntas_todas_con_datos_usuario_renamed.columns)),
      title = f'Total filas {df_preguntas_todas_con_datos_usuario_renamed.shape[0]}'
  )
  fig.show()


In [ ]:
#@markdown Filtros de datos (**Interactivo**)

if not FILTRO_CURSO:
  lista_cursos = obtener_listado_cursos(access_token)

  seleccion_curso = widgets.ToggleButtons(
      options=lista_cursos,
      description='Curso:',
      disabled=False,
      button_style='',
      tooltips=lista_cursos,
  )
  display(seleccion_curso)

if not FILTRO_ENCUESTA:
  seleccion_tipo_encuesta = widgets.ToggleButtons(
      options=TIPOS_DE_ENCUESTA,
      description='Encuesta:',
      disabled=False,
      button_style='',
      tooltips=TIPOS_DE_ENCUESTA,
  )
  # Aplicar estilo CSS para ajustar el ancho del texto

  display(seleccion_tipo_encuesta)
  display(HTML("""
  <style>
  .widget-toggle-button{
      min-width: 300px !important;
  }
  </style>
  """))

In [ ]:
#@markdown Buscar, filtrar y transformar los datos

try:
    FILTRO_CURSO = seleccion_curso.value
except NameError:
    FILTRO_CURSO = FILTRO_CURSO # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

try:
    FILTRO_ENCUESTA = seleccion_tipo_encuesta.value
except NameError:
    FILTRO_ENCUESTA = FILTRO_ENCUESTA # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea



res = obtener_cuestionario(FILTRO_CURSO, access_token, keyname="")
assert len(res) > 0, "Hay un error al obtener la información del cuestionario"
df = limpiar_cuestionario(res)
user_data = obtener_users_info(df['user'].unique(), access_token)
df_user = limpiar_user_info(user_data)

# TOMO LO QUE COINCIDA CON LA CANTIDAD DE PREGUNTAS EN "RAW"
df_ultimo_cuestionario = filtrar_y_eliminar_filas(df, FILTRO_ENCUESTA) # Filtro las respuestas que coincidan con la versión del formulario en la columna "raw"

if not df_ultimo_cuestionario.empty:
  df = df_user.merge(df_ultimo_cuestionario, how='right', left_on='username', right_on='user').drop('user', axis=1)
  df_respuestas = extraer_respuestas(df, FILTRO_ENCUESTA, True, True)

  exclude_list = [
      'points_earned',
      'points_possible',
      'raw',
      'type',
      'respuestas',
      'block_id',
      'grade',
      'nota',
      'Si aceptás compartir tu testimonio, y querés, nos dejarías un link a una fotito tuya en caso de compartirlo en redes o en el sitio? :)'
  ]


  df_preguntas_todas_con_datos_usuario = reemplazar_columnas_por_preguntas(df_respuestas)
  exclude_list = df_preguntas_todas_con_datos_usuario.filter(exclude_list).columns
  df_preguntas_todas_con_datos_usuario = df_preguntas_todas_con_datos_usuario.drop(columns=exclude_list)

  # Chequeamos repetidos por mismas respuestas y mismo email.
  columnas_check_repetido = list(df_preguntas_todas_con_datos_usuario.iloc[:, (-(len(df_preguntas_todas_con_datos_usuario.loc[:, 'datetime':].columns))+1):].columns.values) # Esta línea es para tomar todas las columnas de preguntas
  columnas_check_repetido.append('email')
  df_preguntas_todas_con_datos_usuario = df_preguntas_todas_con_datos_usuario.drop_duplicates(subset=columnas_check_repetido)
  df_preguntas_todas_con_datos_usuario = df_preguntas_todas_con_datos_usuario[~df_preguntas_todas_con_datos_usuario['email'].isin(EMAILS_A_FILTRAR)] # Filtrar las filas donde el correo electrónico no está en la lista

  print(f"Total respuestas después de quitar duplicados y mails de prueba: {df_preguntas_todas_con_datos_usuario.shape[0]}")
else:
  print(f"No hay datos de {FILTRO_ENCUESTA} del curso {FILTRO_CURSO}")

In [ ]:
#@markdown Generación del Dataframe que contiene solo las preguntas y respuestas con la fecha.
columnas_no_preguntas = [
    'username',
    'email',
    'country',
    'last_login',
    'is_active',
    'class_name'
]

df_filtro_columnas_no_preguntas = df_preguntas_todas_con_datos_usuario.filter(columnas_no_preguntas)
df_preguntas_todas_sin_datos_usuario = df_preguntas_todas_con_datos_usuario.drop(columns=df_filtro_columnas_no_preguntas.columns)
df_preguntas_todas_sin_datos_usuario.set_index(['datetime'], inplace=True)
df_preguntas_todas_sin_datos_usuario.index = pd.to_datetime(df_preguntas_todas_sin_datos_usuario.index)


In [ ]:
#@markdown Filtros para graficar (**Interactivo**)

if not (FILTRO_FECHAINI and FILTRO_FECHAINI):
  start_date = df_preguntas_todas_sin_datos_usuario.index.min().replace(hour=0, minute=0, second=0)
  end_date = df_preguntas_todas_sin_datos_usuario.index.max().replace(hour=23, minute=59, second=59)

  # Traducir las fechas a español

  dates_range = pd.date_range(start_date, end_date, freq='D')

  # Traducir las fechas a español y formatearlas directamente
  dates_spanish = [dates.format_date(date, format='medium', locale='es_ES') for date in dates_range]

  # Crear opciones para el SelectionRangeSlider
  options = list(zip(dates_spanish, dates_spanish))
  index = (0, len(options)-1)

  selection_range_slider = widgets.SelectionRangeSlider(
      options=options,
      index=index,
      description='Fechas',
      orientation='horizontal',
      layout={'width': '500px'}
  )

  # Función para traducir las fechas seleccionadas a objetos datetime de pandas
  def translate_to_datetime(index):
      start_date = dates_range[index[0]]
      end_date = dates_range[index[1]]
      return start_date, end_date

  # Función para modificar las fechas seleccionadas
  def modify_selected_dates(change):
      global selected_dates_modificado
      selected_dates = translate_to_datetime(change.new)
      selected_dates_modificado = (selected_dates[0], selected_dates[1] + pd.Timedelta(days=1))

  # Obtener las fechas seleccionadas en formato datetime de pandas
  selected_dates = translate_to_datetime(selection_range_slider.index)
  selected_dates_modificado = (selected_dates[0], selected_dates[1] + pd.Timedelta(days=1))

  # Observar cambios en los índices del slider y llamar a la función para modificar las fechas seleccionadas
  selection_range_slider.observe(modify_selected_dates, names='index')

  items = [selection_range_slider]
  display(widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(1, 200px)")))

In [ ]:
#@markdown #### Respuestas numéricas

try:
    FILTRO_FECHAINI = selected_dates_modificado[0]
except NameError:
    FILTRO_FECHAINI = FILTRO_FECHAINI # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

try:
    FILTRO_FECHAFIN = selected_dates_modificado[1]
except NameError:
    FILTRO_FECHAFIN = FILTRO_FECHAFIN # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

graficar_respuestas_numericas(df_preguntas_todas_sin_datos_usuario, FILTRO_FECHAINI, FILTRO_FECHAFIN)

In [ ]:
#@markdown #### Respuestas categóricas
try:
    FILTRO_FECHAINI = selected_dates_modificado[0]
except NameError:
    FILTRO_FECHAINI = FILTRO_FECHAINI # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

try:
    FILTRO_FECHAFIN = selected_dates_modificado[1]
except NameError:
    FILTRO_FECHAFIN = FILTRO_FECHAFIN # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

graficar_respuestas_categoricas(df_preguntas_todas_sin_datos_usuario, FILTRO_FECHAINI, FILTRO_FECHAFIN)

In [ ]:
#@markdown #### Correlaciones

try:
    FILTRO_FECHAINI = selected_dates_modificado[0]
except NameError:
    FILTRO_FECHAINI = FILTRO_FECHAINI # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

try:
    FILTRO_FECHAFIN = selected_dates_modificado[1]
except NameError:
    FILTRO_FECHAFIN = FILTRO_FECHAFIN # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

graficar_correlaciones(df_preguntas_todas_sin_datos_usuario, FILTRO_FECHAINI, FILTRO_FECHAFIN)

In [ ]:
#@markdown #### Comentarios o sugerencias
try:
    FILTRO_FECHAINI = selected_dates_modificado[0]
except NameError:
    FILTRO_FECHAINI = FILTRO_FECHAINI # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

try:
    FILTRO_FECHAFIN = selected_dates_modificado[1]
except NameError:
    FILTRO_FECHAFIN = FILTRO_FECHAFIN # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

analisis_comentarios(df_preguntas_todas_con_datos_usuario, FILTRO_FECHAINI, FILTRO_FECHAFIN, DUMMY_GPT)

In [ ]:
#@markdown #### Filtro para ver registros completos
filtros_columnas = {}
for columna in df_preguntas_todas_con_datos_usuario.columns:
  columna_texto = columna
  if columna in DF_CONFIGURACION_PREGUNTAS['pregunta'].values:
    columna_texto = DF_CONFIGURACION_PREGUNTAS[DF_CONFIGURACION_PREGUNTAS['pregunta'] == columna]['pregunta_corta'].values[0]
  filtros_columnas[columna_texto] = widgets.Text(
      value='',
      placeholder='Dejar en blanco para evitar filtrar por este campo',
      description=f"{columna_texto}:",
      disabled=False
  )
  display(HTML("""
  <style>
  .widget-label{
      min-width: 400px !important;
  }
  .widget-text input[type="text"]{
      min-width: 300px !important;
  }
  </style>
  """))
  display(filtros_columnas[columna_texto])

In [ ]:
#@markdown #### Visualizar registros completos
try:
    FILTRO_FECHAINI = selected_dates_modificado[0]
except NameError:
    FILTRO_FECHAINI = FILTRO_FECHAINI # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

try:
    FILTRO_FECHAFIN = selected_dates_modificado[1]
except NameError:
    FILTRO_FECHAFIN = FILTRO_FECHAFIN # Lo dejo implicito para que sea más claro a la lectura pero no es necesaria esta línea

filtrar_registros_completos(df_preguntas_todas_con_datos_usuario, filtros_columnas, FILTRO_FECHAINI, FILTRO_FECHAFIN)